In [1]:
%%bash
pip install lxml
#pip install requests
#pip install futures
#pip install jsonpath-rw

In [1]:
import requests
import concurrent.futures

from lxml import html
from IPython.display import JSON,HTML

In [2]:
from jsonpath_rw import jsonpath, parse
import jsonpath_rw_ext as jsonp
import collections
import os

In [3]:
import json
from json import JSONEncoder, JSONDecoder
import pickle

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, unicode, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(str(dct['_python_object']))
    return dct

def save_to_json_file(filename, content):
    temp = json.dumps(content, cls=PythonObjectEncoder)
    return save_to_file(filename, temp)

def save_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return "saved " + filename    

def load_from_json_file(file_name):
    content = load_from_file(file_name)
    result = json.load(content, object_hook=as_python_object)
    return result

def load_from_file(file_name):
    content = open(file_name)
    return content

def as_json(content):
    return json.loads(content)

In [4]:
points = [1, 5, 20, 100, 500]

import sys

def print_progress(i):
    if (i % points[0] == 0):
        if (i % points[1] == 0):
            sys.stdout.write(' ')
            if (i % points[2] == 0):
                sys.stdout.write('  ')
                if (i % points[3] == 0):
                    sys.stdout.write('\n')
                    if (i % points[4] == 0):
                        sys.stdout.write('\n')
        sys.stdout.write('.')
        sys.stdout.flush()                        

In [5]:
s = requests.Session()
s.post('https://en.boardgamearena.com/account/account/login.html', data = {'email' : 'tolar', 'password' : 'here'})
filename = "pr_tables.json"

In [6]:
def load_latest_tables(batch, from_id, from_time):
    url = 'https://en.boardgamearena.com/message/board?type=lastresult&social=false&dojo.preventCache=1537975056450&id=14'+\
        '&page=0&per_page={per_page}&from_time={from_time}&from_id={from_id}'.format(per_page=batch, from_id=from_id, from_time=from_time)
    page = s.get(url).text
    return as_json(page)

In [7]:
def load_batch(amout, from_time=1550000000, from_id=1):
    result = []
    x = load_latest_tables(amout, from_id, from_time)
    for t in x['data']:
        tree = html.fromstring(t['html'])
        ll = tree.xpath('//a/@href')
        ids = [l[l.find('=')+1:] for l in ll]
        (ids[0], ids[1], len(ids)-1, t['timestamp'])
        result.append({
            'table': ids[0], 
            'player': ids[1], 
            'player_amount': len(ids)-1, 
            'timestamp': t['timestamp']
        })
    return result

In [8]:
def table_iterator(stop=30, batch=5000, ts = 1550000000):
#     print('starting with ' + str(dict(stop=stop, batch=batch, ts = ts)))
    for i in range(stop // batch):
        for t in load_batch(batch, ts):
            yield t
        if t:
            ts = t['timestamp']
        print_progress(i)
    for t in load_batch(stop % batch, ts):
        yield t

In [9]:
def get_table_info(tbl):
    url = 'https://en.boardgamearena.com/table/table/tableinfos.html?id={table}&dojo.preventCache=1537970039954'.format(table=tbl)
    page = s.get(url).text
    data = as_json(page)['data']
    return data

In [10]:
def get_table_datas(table, player, version):
    url = 'https://en.boardgamearena.com/archive/replay/{version}/?table={table}&player={player}&comments=1' \
        .format(table=table, player=player, version=version)
    page = s.get(url).text

    tree = html.fromstring(page)
    script = tree.xpath('//div[@id="overall-content"]/script')[0]
    text = script.text_content()

    start = text.find('completesetup')
    end = text.find('\n', start)
    return as_json('[' +text[start+14:end-2]+']')

In [11]:
def get_game_log(table):
    url = 'https://en.boardgamearena.com/archive/archive/logs.html?table={table}&translated=false&dojo.preventCache=1537972617341' \
        .format(table=table)
    page = s.get(url).text
    return as_json(page)

In [12]:
def load_table_datas(table, player):
#     try:
        info = get_table_info(table)
        if not valid(info):
            return None
        datas = get_table_datas(table, player, info['siteversion'])
        log = get_game_log(table)
        return {
            'table': table,
            'info': info,
            'datas': datas,
            'log': log
        }
#     except KeyboardInterrupt:
#         raise
#     except:
#         print('failed to load a table ({}, {}): {}'.format(table, player, sys.exc_info()[0]))

In [13]:
l = [x for x in load_from_json_file(filename) if x['player_amount'] == 4]
len(l)

100616

In [14]:
def stats(total, processed=1, successful=0, min=1e-5, **kwargs):
    print(dict(left  = total - processed,                
               speed = round(processed / (min+1e-5), 1), 
               eta   = round( min*(total-processed)/(processed+1e-5), 2),
               **kwargs))
    return time.time() + 30

In [15]:
def run(func, arg_list, consumer=None, thread_count=10):
    timer_log = start_time = time.time()
    processed = 0
   
    with concurrent.futures.ThreadPoolExecutor(max_workers=thread_count) as executor:
        future_to_table = {executor.submit(func, args) for args in arg_list}
        for future in concurrent.futures.as_completed(future_to_table):
            future_to_table.remove(future)
            result = future.result()
            if consumer:
                consumer(result)
                
            print_progress(processed)
            processed += 1
       
            if (timer_log < time.time()):    
                min = (time.time() - start_time)/60.
                timer_log = stats(len(arg_list), processed, min)

In [16]:
def select_all(key):
    return lambda data: jsonp.match(key, data)

def select_one(key):
    return lambda data: jsonp.match1(key, data)

def contains(key, value):
    return lambda data: jsonp.match1(key, data) == value

In [17]:
def all_of(*filters):
    def fn(data):
        for f in filters:
            if not f(data):
                return False
        return True  
    return fn

In [18]:
def valid(info):
    return all_of(
        contains('$.options."100".value', '0'),
        contains('$.options."101".value', '0'),
        contains('$.options."102".value', '0'),
        contains('$.options."103".value', '0')                      
    )(info)

In [19]:
import os.path

def load_table(t):
    tbl = t['table']
    fn = 'pr/pr_{}.json'.format(tbl)
    if os.path.exists(fn):
        pass
    d = load_table_datas(tbl, t['player'])
    if d:
        save_to_json_file(fn, d)

In [56]:
%%bash
ls pr | wc -l 

308


In [58]:
for i,t in enumerate(l[450:470]):
    fn = 'pr/pr_{}.json'.format(t['table'])
    if not os.path.exists(fn):
        info = get_table_info(t['table'])
        if valid(info):
            print(450+i)

468
469


In [59]:
for i,t in enumerate(l[468:500]):
    print_progress(i)
    load_table(t)

   

.

IndexError: list index out of range

In [ ]:
%%bash
# tar -cvzf pr_4.tgz pr
# tar -cvzf pr_tables.tgz pr_tables.json
# tar -xvzf pr_tables.tgz
# tar -xvzf pr_4.tgz